In [1]:
import numpy as np
import pandas as pd
import optuna
import tensorflow as tf 
import keras
import os

In [2]:
import tensorflow.keras as keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.python.framework import ops
from sklearn.metrics import mean_squared_error



def r2(y_true, y_pred):
    SSE = backend.sum(backend.square(y_pred - y_true))
    SST = backend.sum(backend.square(y_true-backend.mean(y_true)))
    r2 = 1-SSE/SST
    return r2

def create_model(input_length, num_dense_layers, num_dense_nodes, learning_rate, drop_out,nonlinearity):

    model = Sequential()
    model.add(Flatten(input_shape=[input_length, 1]))

    for i in range(int(num_dense_layers)):
        name = 'layer_dense_{0}'.format(i + 1)
        model.add(Dense(num_dense_nodes, activation=nonlinearity, name=name ))
        model.add(Dropout(drop_out)),

    opt = Adam(learning_rate=learning_rate)
    model.add(Dense(1, activation='linear'))
    model.compile(loss="mse",optimizer=opt, metrics=['mse',r2])
    return model

In [11]:
def fitness(params):   
    learning_rate= params.suggest_loguniform('learning_rate', 1e-6, 0.1)
    num_dense_layers= params.suggest_int('num_dense_layers', 1, 6, 1)
    num_dense_nodes= params.suggest_int('num_dense_nodes', 10, 200,2)
    drop_out= params.suggest_uniform('drop_out', 0, 0.5)
    nonlinearity = params.suggest_categorical("AC", ['relu', 'tanh', 'sigmoid', 'linear'])
    input_length = len(train_x[0])
    model = create_model(input_length, num_dense_layers, num_dense_nodes, learning_rate, drop_out,nonlinearity)
    
    history = model.fit(train_x, train_y, batch_size=1000, epochs=100, verbose=0)
    print('train_loss {:.4f}'.format(history.history['loss'][-1]))
    test_pred = model.predict(test_x)
    RMSE = mean_squared_error(test_y,test_pred,squared=False)
    del model  
    keras.backend.clear_session()
    ops.reset_default_graph()

    
    return RMSE

In [13]:


def pred_dataset(file_names):
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    for file_name in file_names:
        temp_data = pd.read_excel(os.path.join(source_path,file_name+'-01-10-2022.xlsx'),skiprows= 1 )
        
        temp_data = temp_data.dropna(axis=0,how = 'any',subset = ["BET","Vt"])#,'Vmic','Vmeso'
        temp_data = temp_data[temp_data['Pressure']>0.1]
        #temp_data = temp_data[temp_data['Vmic']<2]
        index = temp_data['Index'].max()
        test_index= np.random.choice(list(range(index+1)),int(0.2*index),replace=False)
        train_x = temp_data.loc[~temp_data['Index'].isin( test_index)]
        test_x = temp_data.loc[temp_data['Index'].isin(test_index)]
        
        train_df = pd.concat([train_df,train_x],axis=0)
        test_df = pd.concat([test_df,test_x],axis =0)
    return train_df,test_df

In [14]:
train_df,test_df = pred_dataset(['CO2','Methane','Ethane&Ethylene','CFCs',])

In [15]:
#input_feature = ['E','V','L','BET',"Vmeso",'Vt','Vmic','Temp(K)','Pressure']
input_feature = ['E','V','L','BET','Vt','Temp(K)','Pressure']  # break Vt
output = ['Adsorp(mmol/g)']
train_x= train_df[input_feature].values
test_x = test_df[input_feature].values
train_y = train_df[output].values
test_y = test_df[output].values

In [16]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()

train_x = x_scaler.fit_transform(train_df[input_feature])
test_x = x_scaler.transform(test_df[input_feature])

In [17]:
import optuna
study = optuna.create_study(sampler = optuna.samplers.TPESampler(),direction = 'minimize') #NSGAIISampler
study.optimize(fitness, n_trials = 100)
study.best_params

[I 2022-01-31 15:54:37,647] A new study created in memory with name: no-name-faf620f4-fa2c-4f16-8e9f-77588143526e
[I 2022-01-31 15:54:44,352] Trial 0 finished with value: 2.729353218643009 and parameters: {'learning_rate': 0.0003205630041624426, 'num_dense_layers': 2, 'num_dense_nodes': 32, 'drop_out': 0.49582632900342605, 'AC': 'linear'}. Best is trial 0 with value: 2.729353218643009.


train_loss 9.5547


[I 2022-01-31 15:54:49,964] Trial 1 finished with value: 4.967617257509871 and parameters: {'learning_rate': 2.4682523074401476e-06, 'num_dense_layers': 1, 'num_dense_nodes': 126, 'drop_out': 0.31750424478445405, 'AC': 'tanh'}. Best is trial 0 with value: 2.729353218643009.


train_loss 29.0515


[I 2022-01-31 15:54:55,554] Trial 2 finished with value: 1.2733649665022817 and parameters: {'learning_rate': 0.004626539993165899, 'num_dense_layers': 1, 'num_dense_nodes': 106, 'drop_out': 0.18483893550572356, 'AC': 'tanh'}. Best is trial 2 with value: 1.2733649665022817.


...

[I 2022-01-31 16:08:21,808] Trial 98 finished with value: 0.9839368555601241 and parameters: {'learning_rate': 0.018720460032615623, 'num_dense_layers': 4, 'num_dense_nodes': 130, 'drop_out': 0.0503723428960597, 'AC': 'sigmoid'}. Best is trial 84 with value: 0.9561311410594248.


train_loss 1.1409


[I 2022-01-31 16:08:31,726] Trial 99 finished with value: 0.9841436150938696 and parameters: {'learning_rate': 0.03301462729463568, 'num_dense_layers': 4, 'num_dense_nodes': 130, 'drop_out': 0.050229124294004134, 'AC': 'sigmoid'}. Best is trial 84 with value: 0.9561311410594248.


train_loss 1.2052


{'learning_rate': 0.016972496809960123,
 'num_dense_layers': 3,
 'num_dense_nodes': 120,
 'drop_out': 0.018154178211724742,
 'AC': 'sigmoid'}

# Validating the best parameters

In [55]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()

train_x = x_scaler.fit_transform(train_df[input_feature])
test_x = x_scaler.transform(test_df[input_feature])
train_y = y_scaler.fit_transform(train_df[output])
test_y = y_scaler.fit_transform(test_df[output])

In [59]:
input_length = len(train_x[0])
#create_model(input_length, num_dense_layers, num_dense_nodes, learning_rate, drop_out,nonlinearit
best_model = create_model(input_length,2,158,0.024158339300392132,0.014244542201812094,'relu')
best_model.fit(train_x, train_y, validation_data= [test_x,test_y], batch_size=1000, epochs=100, verbose=0)
print(mean_squared_error(test_df[output].values,y_scaler.inverse_transform(best_model.predict(test_x))))


2022-01-13 15:58:27.675964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-13 15:58:28.253652: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-13 15:58:42.318079: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1.0705926831440253


In [60]:
from sklearn.metrics import mean_squared_error,r2_score


from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,\
    GradientBoostingRegressor,ExtraTreesRegressor, BaggingRegressor

    
model_list = [('RF',RandomForestRegressor(n_estimators=50)),('ADB',AdaBoostRegressor(n_estimators=50)),
('GBR',GradientBoostingRegressor(n_estimators=50)),('ETR',ExtraTreesRegressor(n_estimators=50)),
('BGR',BaggingRegressor(ExtraTreesRegressor(n_estimators=50)))]



for model_name, model in model_list:

    model.fit(train_x,train_y)
    test_pre = model.predict(test_x)
    train_error = mean_squared_error(train_y,model.predict(train_x))
    test_error = mean_squared_error(test_y,test_pre)
    print('Algorithm {}, Train_error {}, Test_error {}'.format(model_name,train_error,test_error))
    #plt.scatter(test_y,test_pre)

Algorithm RF, Train_error 0.004373345100766727, Test_error 0.059385398268699596
Algorithm ADB, Train_error 0.3243090585762014, Test_error 0.36334462856269095
Algorithm GBR, Train_error 0.10758119925708831, Test_error 0.1279966950860796
Algorithm ETR, Train_error 1.1292922030014196e-05, Test_error 0.05311842296101357
Algorithm BGR, Train_error 0.004224617945685642, Test_error 0.05291741162874857
